In [32]:
import tensorflow as tf
import keras
import os
import cv2

import numpy as np
from sklearn.metrics import  accuracy_score
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from sklearn.model_selection import train_test_split

from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, Dense, BatchNormalization, Activation, Input, AveragePooling2D, Flatten, Dropout

In [17]:
def SeparableConv(x, num_filters, strides, alpha = 1.0):
    x = DepthwiseConv2D(kernel_size = 3, padding = 'same')(x)
    x = BatchNormalization(momentum = 0.9997)(x)
    x = Activation('relu')(x)
    x = Conv2D(np.floor(num_filters * alpha), kernel_size = (1, 1), strides = strides, use_bias = False, padding = 'same')(x)
    x = BatchNormalization(momentum = 0.9997)(x)
    x = Activation('relu')(x)
    
    return x

In [22]:
def Conv(x, num_filters, kernel_size, strides = 1, alpha = 1.0):
    x = Conv2D(np.floor(num_filters * alpha), kernel_size = (1, 1), strides = strides, use_bias = False, padding = 'same')(x)
    x = BatchNormalization(momentum = 0.9997)(x)
    x = Activation('relu')(x)
    return x

In [23]:
num_classes = 2
inputs = Input(shape = (224, 224, 3))

In [26]:
x = Conv(inputs, num_filters = 32, kernel_size = 3, strides = 2)
x = SeparableConv(x, num_filters=32, strides=1)


x = Conv(x, num_filters = 64, kernel_size = 1)
x = SeparableConv(x, num_filters=64, strides=2)

x = Conv(x, num_filters = 128, kernel_size = 1)
x = SeparableConv(x, num_filters=128, strides=1)

x = Conv(x, num_filters = 128, kernel_size = 1)
x = SeparableConv(x, num_filters=128, strides=2)


x = Conv(x, num_filters = 256, kernel_size = 1)
x = SeparableConv(x, num_filters=256, strides=1)

x = Conv(x, num_filters = 256, kernel_size = 1)
x = SeparableConv(x, num_filters=256, strides=2)


x = Conv(x, num_filters = 512, kernel_size = 1)

for i in range(5):
    x = SeparableConv(x, num_filters=512, strides=1)
    x = Conv(x, num_filters = 512, kernel_size = 1)
    
x = SeparableConv(x, num_filters=512, strides=1)

x = Conv(x, num_filters = 1024, kernel_size = 1)

x = AveragePooling2D(pool_size = (7, 7))(x)
x = Flatten()(x)

outputs = Dense(num_classes, activation = 'softmax')(x)
outputs = Dropout(0.2)(outputs)
outputs = Activation('softmax')(outputs)

In [29]:
model = Model(inputs, outputs)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer =  Adam(0.001),
             metrics = ['acc'])

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_57 (Conv2D)          (None, 112, 112, 32)      96        
                                                                 
 batch_normalization_83 (Bat  (None, 112, 112, 32)     128       
 chNormalization)                                                
                                                                 
 activation_83 (Activation)  (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d_29 (Depthw  (None, 112, 112, 32)     320       
 iseConv2D)                                                      
                                                                 
 batch_normalization_84 (Bat  (None, 112, 112, 32)     128   

In [31]:
datasets_directory  ="datasets/"
categories = ['cats', 'dogs']
img_size = 224

In [33]:
image_augmentation = Sequential([RandomFlip('horizontal_and_vertical'),
                               RandomRotation(0.2)])

In [36]:
trainingDatasets = []
def create_trainingDatasets():
    for i in categories:
        path = os.path.join(datasets_directory, i)
        class_number = categories.index(i)
        
        for img in os.listdir(path):
            imageArray = cv2.imread(os.path.join(path, img))
            img_size = 224
            modifiedArray = cv2.resize(imageArray, (img_size, img_size))
            modifiedArray = tf.expand_dims(modifiedArray, 0)
            
        for  i in range(1000):
            imgAugment = image_augmentation(modifiedArray)
            trainingDatasets.append([modifiedArray, class_number])
            
create_trainingDatasets()

In [37]:
print(len(trainingDatasets))

2000


In [38]:
XDatasets = []
yDatasets = []

for feature, labels in trainingDatasets:
    XDatasets.append(feature)
    yDatasets.append(labels)

In [40]:
XDatasets = np.array(XDatasets).reshape(-1, img_size, img_size,3)
yDatasets = np.array(yDatasets).reshape((-1, 1))

In [41]:
XDatasets = XDatasets.astype('float32')
XDatasets /=255

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(XDatasets, yDatasets, test_size=0.2, random_state=4)

In [45]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test), shuffle = True)

Epoch 1/10
50/50 [==============================] - 29s 422ms/step - loss: 0.3453 - acc: 0.8250 - val_loss: 0.6944 - val_acc: 0.5100
Epoch 2/10
50/50 [==============================] - 20s 405ms/step - loss: 0.3388 - acc: 0.8263 - val_loss: 0.7033 - val_acc: 0.5100
Epoch 3/10
50/50 [==============================] - 20s 402ms/step - loss: 0.3465 - acc: 0.8125 - val_loss: 0.7146 - val_acc: 0.5100
Epoch 4/10
50/50 [==============================] - 20s 391ms/step - loss: 0.3347 - acc: 0.8425 - val_loss: 0.7266 - val_acc: 0.5100
Epoch 5/10
50/50 [==============================] - 20s 410ms/step - loss: 0.3388 - acc: 0.8250 - val_loss: 0.7379 - val_acc: 0.5100
Epoch 6/10
50/50 [==============================] - 20s 406ms/step - loss: 0.3416 - acc: 0.8144 - val_loss: 0.7486 - val_acc: 0.5100
Epoch 7/10
50/50 [==============================] - 19s 379ms/step - loss: 0.3352 - acc: 0.8331 - val_loss: 0.7583 - val_acc: 0.5100
Epoch 8/10
50/50 [==============================] - 20s 393ms/step - 